In [ ]:
import requests
import pandas as pd
import math # Untuk math.ceil
import os # Untuk memeriksa apakah file sudah ada

# --- KONFIGURASI ---
API_KEY = "............................." # Ganti dengan API Key Anda yang valid jika ini tidak berfungsi
BASE_URL = "https://api.elsevier.com/content/search/scopus"
MAX_RESULTS_PER_REQUEST = 25 # Batas aman untuk Scopus Search API 'count' parameter

# --- FUNGSI UNTUK MENGAMBIL DATA ---
def search_scopus_articles(base_query, start_year=None, end_year=None,
                           total_articles_to_fetch=100, batch_size=MAX_RESULTS_PER_REQUEST):
    """
    Mencari artikel di Scopus secara berulang (pagination)
    berdasarkan query, rentang tahun (opsional), hingga jumlah artikel yang diinginkan.
    """
    headers = {
        "X-ELS-APIKey": API_KEY,
        "Accept": "application/json"
    }

    final_query_string = base_query
    year_query_parts = []

    if start_year:
        try:
            year_query_parts.append(f"PUBYEAR > {int(start_year) - 1}") # Inklusif start_year
        except ValueError:
            print(f"Peringatan: Tahun mulai '{start_year}' tidak valid. Filter tahun mulai akan diabaikan.")

    if end_year:
        try:
            year_query_parts.append(f"PUBYEAR < {int(end_year) + 1}") # Inklusif end_year
        except ValueError:
            print(f"Peringatan: Tahun akhir '{end_year}' tidak valid. Filter tahun akhir akan diabaikan.")

    if year_query_parts:
        year_filter_string = " AND ".join(year_query_parts)
        final_query_string = f"({base_query}) AND ({year_filter_string})"

    all_articles_collected = []
    current_start_index = 0
    total_results_available_from_api = 0

    print(f"Mulai mengambil artikel untuk query: {final_query_string}")
    print(f"Target total artikel: {total_articles_to_fetch}, Ukuran batch per permintaan: {batch_size}")

    while True:
        if len(all_articles_collected) >= total_articles_to_fetch:
            print(f"Telah mengumpulkan {len(all_articles_collected)} artikel, target {total_articles_to_fetch} tercapai.")
            break

        if total_results_available_from_api > 0 and current_start_index >= total_results_available_from_api:
            print(f"Indeks awal ({current_start_index}) melebihi total artikel tersedia dari API ({total_results_available_from_api}). Menghentikan pengambilan.")
            break

        params = {
            "query": final_query_string,
            "count": batch_size,
            "start": current_start_index,
            "sort": "-citedby-count"
        }

        print(f"Mengirim permintaan ke Scopus: batch ke-{ (current_start_index // batch_size) + 1 }, mulai dari indeks {current_start_index}, jumlah {batch_size}")

        response = None
        try:
            response = requests.get(BASE_URL, headers=headers, params=params)
            response.raise_for_status()
            data = response.json()

            search_results_data = data.get('search-results', {})
            articles_in_batch = search_results_data.get('entry', [])

            if not articles_in_batch:
                print("Tidak ada artikel lagi yang ditemukan dari API pada batch ini.")
                break

            all_articles_collected.extend(articles_in_batch)
            print(f"Berhasil mengambil {len(articles_in_batch)} artikel.")
            print(f"Total terkumpul: {len(all_articles_collected)}.")

            if total_results_available_from_api == 0:
                try:
                    total_results_available_from_api = int(search_results_data.get('opensearch:totalResults', 0))
                    print(f"API melaporkan total {total_results_available_from_api} artikel tersedia untuk query ini.")
                    if total_articles_to_fetch > total_results_available_from_api:
                        print(f"Peringatan: Anda meminta {total_articles_to_fetch} artikel, "
                              f"tetapi API hanya menyediakan {total_results_available_from_api}. Akan diambil semaksimal mungkin.")
                        total_articles_to_fetch = total_results_available_from_api
                except ValueError:
                    print("Peringatan: Tidak dapat membaca total hasil dari API.")

            current_start_index += batch_size

        except requests.exceptions.RequestException as e:
            print(f"Error saat request API: {e}")
            if response is not None:
                print(f"Response status code: {response.status_code}")
                print(f"Response content: {response.text}")
            break
        except ValueError:
            print(f"Error: Respons bukan JSON valid.")
            if response is not None:
                print(f"Response status code: {response.status_code}")
                print(f"Response content: {response.text}")
            else:
                print("No response received.")
            break

        if len(all_articles_collected) >= total_articles_to_fetch:
            print(f"Target {total_articles_to_fetch} artikel tercapai setelah batch terakhir.")
            break
    return all_articles_collected[:total_articles_to_fetch]


# --- FUNGSI UNTUK MEMPROSES DAN MENAMPILKAN DATA ---
def display_articles(articles):
    if not articles:
        print("Tidak ada artikel ditemukan atau terjadi error pada pengambilan data.")
        return None

    processed_articles = []
    for article in articles:
        title = article.get('dc:title', 'N/A')
        creator = article.get('dc:creator', 'N/A')
        publication_name = article.get('prism:publicationName', 'N/A')
        cover_date = article.get('prism:coverDate', 'N/A')
        cited_by_count = article.get('citedby-count', 'N/A')
        doi = article.get('prism:doi')
        eid = article.get('eid', 'N/A')

        access_link = 'N/A'
        if doi:
            access_link = f"https://doi.org/{doi}"
        else:
            scopus_page_links = [
                link_item.get('@href')
                for link_item in article.get('link', [])
                if link_item.get('@ref') == 'scopus' and link_item.get('@href')
            ]
            if scopus_page_links:
                access_link = scopus_page_links[0]

        processed_articles.append({
            "Judul": title,
            "Penulis Utama": creator,
            "Jurnal/Sumber": publication_name,
            "Tanggal Publikasi": cover_date,
            "Jumlah Sitasi": cited_by_count,
            "DOI": doi if doi else 'N/A',
            "EID": eid,
            "Link Akses": access_link
        })

    if not processed_articles:
        print("Data artikel yang diproses kosong.")
        return None

    df = pd.DataFrame(processed_articles)

    print(f"\n--- Hasil Pencarian {len(df)} Artikel Kamu ---")
    if not df.empty:
        pd.set_option('display.max_colwidth', None)
        print(df.to_string(index=False))
    else:
        print("DataFrame kosong, tidak ada data untuk ditampilkan.")
    return df

# --- FUNGSI UNTUK MENYIMPAN KE CSV ---
def save_to_csv(dataframe, default_filename="hasil_pencarian_scopus.csv"):
    """
    Meminta pengguna untuk menyimpan DataFrame ke file CSV.
    """
    if dataframe is None or dataframe.empty:
        print("Tidak ada data untuk disimpan ke CSV.")
        return

    while True:
        choice = input("Apakah Anda ingin menyimpan hasil ini ke file CSV? y=YA / n=TIDAK, Silahkan pilih (y/n): ").strip().lower()
        if choice in ['y', 'n', 'yes', 'no']:
            break
        else:
            print("Pilihan tidak valid. Silakan masukkan 'y' atau 'n'.")

    if choice in ['y', 'yes']:
        filename_input = input(f"Berikan nama file CSV yang ingin anda simpan (default: {default_filename}): ").strip()
        if not filename_input:
            filename = default_filename
        else:
            filename = filename_input if filename_input.endswith(".csv") else filename_input + ".csv"

        # Periksa apakah file sudah ada dan minta konfirmasi untuk menimpa
        if os.path.exists(filename):
            overwrite_choice = input(f"File '{filename}' sudah ada. Timpa file? (y/n): ").strip().lower()
            if overwrite_choice not in ['y', 'yes']:
                print("Penyimpanan dibatalkan oleh pengguna.")
                return

        try:
            dataframe.to_csv(filename, index=False, encoding='utf-8-sig') # utf-8-sig untuk kompatibilitas Excel yang lebih baik
            print(f"Data berhasil disimpan ke {filename}")
        except Exception as e:
            print(f"Terjadi error saat menyimpan file CSV: {e}")
    else:
        print("Hasil tidak disimpan ke file CSV.")


# --- CONTOH PENGGUNAAN ---
if __name__ == "__main__":
    if API_KEY == "MASUKKAN_API_KEY_SCOPUS_ANDA_DI_SINI":
        print("Error: Silakan masukkan API Key Scopus Anda pada variabel API_KEY.")
    elif API_KEY == "..............................":
         print("DEEP RESEARCH JOURNAL SCOPUS v.3 - Developed By SYAHRUL. G")

    keyword_query = 'SYARIAH STOCK PRICES' # TULIS JUDUL JURNAL YANG INGIN DICARI DISINI!

    search_start_year = 2017 # TULIS RENTANG TAHUN AWAL YANG INGIN DICARI
    search_end_year = 2026 # TULIS RENTANG TAHUN AKHIR YANG INGIN DICARI

    total_articles_to_retrieve = 500 # JUMLAH ARTIKEL YANG INGIN DITEMUKAN

    log_message = f"\nMencari artikel dengan kata: '{keyword_query}'"
    if search_start_year is not None and search_end_year is not None:
        log_message += f" dari tahun {search_start_year} hingga {search_end_year}"
    elif search_start_year is not None:
        log_message += f" dari tahun {search_start_year}"
    elif search_end_year is not None:
        log_message += f" hingga tahun {search_end_year}"
    else:
        log_message += " (tanpa filter tahun)"
    print(log_message)
    print(f"Akan mencoba mengambil hingga {total_articles_to_retrieve} artikel.")

    articles_data = search_scopus_articles(
        keyword_query,
        start_year=search_start_year,
        end_year=search_end_year,
        total_articles_to_fetch=total_articles_to_retrieve,
        batch_size=MAX_RESULTS_PER_REQUEST
    )

    if articles_data:
        print(f"\nTotal artikel yang berhasil dikumpulkan: {len(articles_data)}")
        displayed_df = display_articles(articles_data)
        if displayed_df is not None and not displayed_df.empty:
            print(f"Menampilkan {len(displayed_df)} artikel.")
            # Tawarkan untuk menyimpan ke CSV
            save_to_csv(displayed_df)
        elif displayed_df is None or displayed_df.empty:
             print("Tidak ada data untuk ditampilkan atau disimpan.")
    else:
        print("Proses pengambilan atau penampilan data artikel tidak berhasil sepenuhnya.")